In [1]:
# This takes some well-behaved Altair Fizeau frames, derotates them, and takes the median

# created 2018 Oct. 14 by E.S.

In [1]:
import urllib
import numpy as np
import matplotlib.pyplot as plt

import PynPoint

from PynPoint import Pypeline
from PynPoint.IOmodules.FitsReading import FitsReadingModule
from PynPoint.IOmodules.FitsWriting import FitsWritingModule
from PynPoint.IOmodules.TextReading import ParangReadingModule
'''
from PynPoint.Util.Multiprocessing import LineProcessingCapsule, \
                                          PcaMultiprocessingCapsule
'''
from PynPoint.ProcessingModules import PSFpreparationModule, \
                                       PcaPsfSubtractionModule, \
                                       ImageResizing, \
                                       ContrastCurveModule, \
                                       StackingAndSubsampling

In [2]:
working_place = "./pynpoint_experimentation_altair/working_place/"
input_place = "./pynpoint_experimentation_altair/input_place/escrow/"
output_place = "./pynpoint_experimentation_altair/output_place/"

#url = urllib.URLopener()
#url.retrieve("https://people.phys.ethz.ch/~stolkert/BetaPic_NACO_Mp.hdf5",
#             input_place+"BetaPic_NACO_Mp.hdf5")

pipeline = Pypeline(working_place_in=working_place,
                    input_place_in=input_place,
                    output_place_in=output_place)

# now a *.ini file has been generated (this includes the PIXSCALE), if no pre-existing one was there

Initiating PynPoint... [DONE]


In [3]:
# read in science FITS files

read_science = FitsReadingModule(name_in="read_science",
                                 input_dir=None,
                                 image_tag="science",
                                 check=True)

pipeline.add_module(read_science)

In [4]:
# derotate frames and take median across them

derotate_frames = StackingAndSubsampling.DerotateAndStackModule(name_in='derotate_stack', 
                                         image_in_tag='science', 
                                         image_out_tag='science_derotated', 
                                         derotate=True, 
                                         stack=None, 
                                         extra_rot=0.0)

pipeline.add_module(derotate_frames)

In [5]:
# write out

write_test = FitsWritingModule(file_name="junk_derotated_cube.fits",
                              name_in="write_test",
                              output_dir=output_place,
                              data_tag="science_derotated")

pipeline.add_module(write_test)

In [6]:
# run pipeline

pipeline.run()

Validating Pypeline... [DONE]


/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:172: UserWarning: Static attribute DIT (=ESO DET DIT) not found in the FITS header.
  % (item[0], fitskey))
/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:172: UserWarning: Static attribute LATITUDE (=ESO TEL GEOLAT) not found in the FITS header.
  % (item[0], fitskey))
/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:172: UserWarning: Static attribute LONGITUDE (=ESO TEL GEOLON) not found in the FITS header.
  % (item[0], fitskey))
/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:207: UserWarning: Non-static attribute EXP_NO (=ESO DET EXP NO) not found in the FITS header.
  "header." % (item[0], fitskey))
/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:207: UserWarning: Non-static attribute ND

Running FitsReadingModule... [DONE]
Running DerotateAndStackModule... [DONE]
Running FitsWritingModule... [DONE]


In [7]:
residuals = pipeline.get_data("residuals")
pixscale = pipeline.get_attribute("science", "PIXSCALE")
contrastdata = pipeline.get_attribute("contrast_landscape")

size = pixscale*np.size(residuals, 1)/2.

In [8]:
plt.imshow(residuals[0, ], origin='lower', extent=[size, -size, -size, size])
plt.title("Altair test")
plt.xlabel('R.A. offset [arcsec]', fontsize=12)
plt.ylabel('Dec. offset [arcsec]', fontsize=12)
plt.colorbar()
#plt.show()
plt.savefig(output_place+"residuals.png")